In [119]:
import numpy as np, random, math, operator, pandas as pd, matplotlib.pyplot as plt

In [144]:
random.seed(42)

In [120]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

In [121]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
    
    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness
    
    


In [168]:
class GAAgent():
    """
    Hyperparameters: 
        intensification: eliteSize
        diversification: popSize, mutationRate
    """
    
    def __init__(self, popSize, eliteSize, mutationRate):
        self.popSize = popSize
        self.eliteSize = eliteSize
        self.mutationRate = mutationRate
    
    
    def createRoute(self, cityList):
        return random.sample(cityList, len(cityList))

    def initialPopulation(self, popSize, cityList):
        return [self.createRoute(cityList) for i in range(popSize)]

    def rankRoutes(self, population):
        fitnessResults = {i:Fitness(population[i]).routeFitness() for i in range(len(population))}
        return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

    def selection(self, popRanked, eliteSize):
        df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
        df['cum_sum'] = df.Fitness.cumsum()
        df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
        
        selectionResults = [popRanked[i][0] for i in range(eliteSize)]
        for i in range(0, len(popRanked) - eliteSize):
            pick = 100*random.random()
            for i in range(0, len(popRanked)):
                if pick <= df.iat[i,3]:
                    selectionResults.append(popRanked[i][0])
                    break
                    
        return selectionResults


    def matingPool(self, population, selectionResults):
        matingpool = [population[selectionResults[i]] for i in range(len(selectionResults))]
        return matingpool


    def breed(self, parent1, parent2):
        child = []
        childP1 = []
        childP2 = []

        geneA = int(random.random() * len(parent1))
        geneB = int(random.random() * len(parent1))

        startGene = min(geneA, geneB)
        endGene = max(geneA, geneB)

        for i in range(startGene, endGene):
            childP1.append(parent1[i])

        childP2 = [item for item in parent2 if item not in childP1]

        child = childP1 + childP2
        return child

    def breedPopulation(self, matingpool, eliteSize):
        children = []
        length = len(matingpool) - eliteSize
        pool = random.sample(matingpool, len(matingpool))

        for i in range(0,eliteSize):
            children.append(matingpool[i])

        for i in range(0, length):
            child = self.breed(pool[i], pool[len(matingpool)-i-1])
            children.append(child)
        return children


    def mutate(self, individual, mutationRate):
        for swapped in range(len(individual)):
            if(random.random() < mutationRate):
                swapWith = int(random.random() * len(individual))

                city1 = individual[swapped]
                city2 = individual[swapWith]

                individual[swapped] = city2
                individual[swapWith] = city1
        return individual


    def mutatePopulation(self, population, mutationRate):
        mutatedPop = []

        for ind in range(0, len(population)):
            mutatedInd = self.mutate(population[ind], mutationRate)
            mutatedPop.append(mutatedInd)
        return mutatedPop

    def _step(self, currentGen, eliteSize, mutationRate):
        popRanked = self.rankRoutes(currentGen)
        selectionResults = self.selection(popRanked, eliteSize)
        matingpool = self.matingPool(currentGen, selectionResults)
        children = self.breedPopulation(matingpool, eliteSize)
        nextGeneration = self.mutatePopulation(children, mutationRate)
        return nextGeneration

    def search(self, population, generations):
        
        pop = self.initialPopulation(self.popSize, population)
        for i in range(0, generations):
            pop = self._step(pop, self.eliteSize, self.mutationRate)
        bestRouteIndex = self.rankRoutes(pop)[0][0]
        bestRoute = pop[bestRouteIndex]
        
        return bestRoute, self.rankRoutes(pop)[0][1]

In [148]:
def create_cityList(size=25):
    random.seed(42)
    return [City(x=int(random.random() * 200), y=int(random.random() * 200)) for i in range(size)]

In [156]:
cityList = create_cityList(10)

In [169]:
GAA = GAAgent(popSize=100, eliteSize=20, mutationRate=0.01)

print(GAA.search(population=cityList, generations=500))

([(161,1), (178,17), (147,135), (129,108), (44,117), (43,101), (5,39), (55,44), (84,5), (127,5)], 0.0018935500399828603)


In [77]:
class RSAgent():
    
    def __init__(self, chemin_init, t_init, t_min, coeff):
        self.n = len(chemin_init)
        self.chemin_init = chemin_init
        self.t_init = t_init
        self.t_min = t_min
        self.coeff = coeff
    
    def _energie(self, chemin, dist):
        """
        Compute energy of the chemin = sum of dist
        """
        
        distance = 0

        for i in range(self.n-1):
            distance += dist[chemin[i]][chemin[i+1]]

        return distance + dist[chemin[self.n-1]][chemin[0]]
    
    def _step(self, chemin, E, temp):
        """
        Does a step of the RS algorithm
        
        Inputs:
            chemin
            E
            temp
            
        Returns:
            chemin
            E
            temp
        """
        
        sommet1 = random.randint(0, self.n-1)
        sommet2 = random.randint(0, self.n-1)

        chemin_b = chemin[:]

        temporaire = chemin_b[sommet1]
        chemin_b[sommet1] = chemin[sommet2]
        chemin_b[sommet2] = temporaire
        E_b = self._energie(chemin_b, dist)

        if (E_b < E) or (random.random() < math.exp((E - E_b) / temp)):
            chemin = chemin_b
            E = E_b

        temp *= self.coeff
        
        return chemin, E, temp
        
    
    
    def search(self, dist):
        """
        Input:
            distance n x n matrix
            
        Returns:
            chemin: optimal path found
            E     : energy of that path (sum of distances)
        """
        
        n = len(dist)
        chemin = self.chemin_init
        E = self._energie(chemin, dist)
        temp = self.t_init
        
        while temp > self.t_min:
            
            
            chemin, E, temp = self._step(chemin, E, temp)
            
        return chemin, E
        

In [91]:
def test_RSA(print_chemin = False):

    dist = [[0, 780, 320, 580, 480, 660], 
            [780, 0, 700, 460, 300, 200], 
            [320, 700, 0, 380, 820, 630], 
            [580, 460, 380, 0, 750, 310], 
            [480, 300, 820, 750, 0, 500], 
            [600, 200, 630, 310, 500, 0]]
    
    chemin_naif = [i for i in range(len(dist))]

    RSA = RSAgent(chemin_init = chemin_naif, t_init = 100, t_min = 1, coeff = 0.99)
    
    chemin, E = RSA.search(dist)
    
    if print_chemin: print(chemin)
    
    assert E == 1990

In [94]:
test_RSA()

In [ ]:
class GAAgent():
    
    
    